In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader

from utils.data_utils import trans_list
from utils.draw_utils import draw_predict_skeleton, draw_mask_skeleton
from utils.pose_seg_dataset import PoseSegDataset

1. 载入实验数据

In [1]:
def load_exp_data(res_npz_path):
    data = np.load(res_npz_path, allow_pickle=True)
    args, output_arr, rec_loss_arr = data['args'].tolist(), data['output_arr'], data['rec_loss_arr']
    # load test dataset
    transform_list = trans_list[:args.num_transform]

    dataset_args = {'transform_list': transform_list, 'debug': args.debug, 'headless': args.headless,
                    'scale': args.norm_scale, 'scale_proportional': args.prop_norm_scale, 'seg_len': args.seg_len,
                    'return_indices': True, 'return_metadata': True, 'hr': args.hr}

    loader_args = {'batch_size': args.batch_size, 'num_workers': args.num_workers, 'pin_memory': True}

    dataset_args['seg_stride'] = 1  # No strides for test set
    dataset_args['train_seg_conf_th'] = 0.0
    dataset = PoseSegDataset(args.pose_path['test'], **dataset_args)
    loader = DataLoader(dataset, **loader_args, shuffle=False)
    return args, output_arr, rec_loss_arr, loader, dataset

In [3]:
npz_path = './data/exp_dir/res_7062_.npz'
args, output_arr, rec_loss_arr, loader, dataset = load_exp_data(npz_path)

In [4]:
from utils.scoring_utils import *
from scipy.ndimage import gaussian_filter1d


2. 计算各scene clip的正常样本异常值 异常样本异常值

In [6]:
score_vals = np.array(rec_loss_arr)
gt_arr, scores_arr, score_ids_arr, metadata_arr = get_dataset_scores_by_rec_add_pre_score(score_vals, dataset.metadata, dataset.person_keys, None, None, args)
scores_arr

Scoring 107 clips


[array([0.00036109, 0.00029964, 0.00031599, 0.00041096, 0.00039148,
        0.00054267, 0.00636639, 0.00784148, 0.00927897, 0.01276438,
        0.00917324, 0.01055911, 0.00931696, 0.01326969, 0.01512578,
        0.01590972, 0.01480548, 0.01193554, 0.0152498 , 0.0203564 ,
        0.03262127, 0.03466678, 0.03425451, 0.0318997 , 0.02814004,
        0.03704423, 0.04932433, 0.0546497 , 0.05998329, 0.04386361,
        0.02566166, 0.02659407, 0.02251561, 0.02906528, 0.01728551,
        0.02179988, 0.01727467, 0.0157207 , 0.01668707, 0.01688947,
        0.01686047, 0.02133991, 0.03130384, 0.02916286, 0.03547913,
        0.02522459, 0.03078019, 0.04001536, 0.03217149, 0.04077937,
        0.04345302, 0.04648035, 0.05350033, 0.04361039, 0.03194947,
        0.03151209, 0.02377102, 0.02408857, 0.032215  , 0.02864933,
        0.02491015, 0.02733203, 0.02313665, 0.02841869, 0.01829362,
        0.03096515, 0.02907372, 0.02760863, 0.0380899 , 0.01512052,
        0.0205986 , 0.01530806, 0.01488315, 0.01

In [7]:
normalized_scores_arrs = normalize_scores(scores_arr)
normalized_scores_arrs

[array([0.00487885, 0.00404858, 0.00426949, 0.00555269, 0.00528955,
        0.00733234, 0.08601974, 0.10595043, 0.12537315, 0.17246644,
        0.12394468, 0.14266987, 0.12588647, 0.179294  , 0.20437262,
        0.2149648 , 0.20004485, 0.16126761, 0.20604826, 0.27504627,
        0.44076364, 0.46840161, 0.46283123, 0.43101416, 0.38021533,
        0.5005246 , 0.6664477 , 0.73840173, 0.81046677, 0.59266501,
        0.34672853, 0.35932687, 0.30422065, 0.39271672, 0.23355391,
        0.29455004, 0.23340746, 0.21241094, 0.22546802, 0.22820272,
        0.22781094, 0.28833504, 0.42296315, 0.39403515, 0.47937774,
        0.34082314, 0.41588782, 0.54066921, 0.43468646, 0.55099218,
        0.58711722, 0.62802121, 0.72287187, 0.58924361, 0.43168665,
        0.42577695, 0.3211831 , 0.32547369, 0.43527434, 0.38709656,
        0.33657449, 0.36929788, 0.3126118 , 0.38398036, 0.24717498,
        0.41838694, 0.39283083, 0.37303518, 0.5146532 , 0.20430156,
        0.27831883, 0.20683546, 0.20109427, 0.25

guassia平滑前

In [ ]:
def score_stat(gt_arr, scores_arr, metadata_arr, smooth = False, sigma = 40):
    pose_neg_score_mean_arr = []

    for gt, score, meta in zip(gt_arr, scores_arr, metadata_arr):
        pos_idx = np.where(gt==0)[0]
        neg_idx = np.where(gt==1)[0]
        if smooth:
            score = gaussian_filter1d(score, sigma)
            
        pos_score_mean = np.mean(score[pos_idx])
        neg_score_mean = np.mean(score[neg_idx])
        if np.isnan(pos_score_mean)  or np.isnan(neg_score_mean):
            print(meta)
            continue
        pose_neg_score_mean_arr.append([pos_score_mean, neg_score_mean])
    total_pos_mean, total_neg_mean = np.mean(pose_neg_score_mean_arr,axis=0)
    return pose_neg_score_mean_arr, total_pos_mean, total_neg_mean


guassia平滑后

In [ ]:
s_pose_neg_score_mean_arr, s_total_pos_mean, s_total_neg_mean = score_stat(gt_arr, scores_arr, metadata_arr, smooth = True, sigma=5)
s_pose_neg_score_mean_arr, s_total_pos_mean, s_total_neg_mean

In [ ]:
pose_neg_score_mean_arr, total_pos_mean, total_neg_mean = score_stat(gt_arr, scores_arr, metadata_arr, smooth = False, sigma=10)
pose_neg_score_mean_arr, total_pos_mean, total_neg_mean

In [ ]:
n_pose_neg_score_mean_arr, n_total_pos_mean, n_total_neg_mean = score_stat(gt_arr, normalized_scores_arrs, metadata_arr, smooth = False, sigma=10)
n_pose_neg_score_mean_arr, n_total_pos_mean, n_total_neg_mean

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
x = np.arange(5)
labels = metadata_arr[:5]
n_pose_neg_score_mean_arr = np.array(n_pose_neg_score_mean_arr)
normal = n_pose_neg_score_mean_arr[:5,0]
abnormal = n_pose_neg_score_mean_arr[:5,1]
# gap = 
total_width, n = 0.8, 2
width = total_width / n
x = x - (total_width - width) / 2

plt.bar(x, normal,  width=width, label='normal')
plt.xticks(x, labels=labels)
plt.bar(x + width, abnormal, width=width, label='abnormal')
plt.legend()
plt.show()

不同场景的差距图

实验不同smooth方法
